In [1]:
import xarray as xr
from glob import glob
import sys
import pandas as pd
import otps
import os
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
from datetime import timedelta
import numpy as np
import scipy
from scipy import ndimage as nd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

import datacube
from datacube.utils import geometry
from datacube.utils.geometry import CRS

def lag_linregress_3D(x, y, lagx=0, lagy=0, first_dim='time'):
    
    """
    Takes two xr.Datarrays of any dimensions (input data could be a 1D time series, or for example, have 
    three dimensions e.g. time, lat, lon), and return covariance, correlation, regression slope and intercept, 
    p-value, and standard error on regression between the two datasets along their aligned first dimension.  
    
    Datasets can be provided in any order, but note that the regression slope and intercept will be calculated
    for y with respect to x.
    
    Parameters
    ----------    
    x, y : xarray DataArray
        Two xarray DataArrays with any number of dimensions, both sharing the same first dimension
    lagx, lagy : int, optional
        Optional integers giving lag values to assign to either of the data, with lagx shifting x, and lagy 
        shifting y with the specified lag amount. 
    first_dim : str, optional
        An optional string giving the name of the first dimension on which to align datasets. The default is
        'time'.
    
    Returns
    -------
    cov, cor, slope, intercept, pval, stderr : xarray DataArray
        Covariance, correlation, regression slope and intercept, p-value, and standard error on 
        regression between the two datasets along their aligned first dimension.  

    """ 
    
    #1. Ensure that the data are properly alinged to each other. 
    x, y = xr.align(x, y)
    
    #2. Add lag information if any, and shift the data accordingly
    if lagx != 0:
        
        # If x lags y by 1, x must be shifted 1 step backwards. But as the 'zero-th' value is nonexistant, xr 
        # assigns it as invalid (nan). Hence it needs to be dropped:
        x = x.shift(**{first_dim: -lagx}).dropna(dim=first_dim)
        
        # Next re-align the two datasets so that y adjusts to the changed coordinates of x:
        x,y = xr.align(x, y)

    if lagy!=0:
        
        y = y.shift(**{first_dim: -lagy}).dropna(dim=first_dim)
        x, y = xr.align(x, y)
 
    #3. Compute data length, mean and standard deviation along time axis for further use: 
    n = y.notnull().sum(dim=first_dim)
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd = x.std(axis=0)
    ystd = y.std(axis=0)
    
    #4. Compute covariance along first axis
    cov = np.sum((x - xmean) * (y - ymean), axis=0) / (n)
    
    #5. Compute correlation along time axis
    cor = cov / (xstd * ystd)
    
    #6. Compute regression slope and intercept:
    slope = cov / (xstd**2)
    intercept = ymean - xmean*slope  
    
    #7. Compute P-value and standard error
    #Compute t-statistics
    tstats = cor*np.sqrt(n - 2) / np.sqrt(1 - cor**2)
    stderr = slope/tstats
    
    from scipy.stats import t
    pval = t.sf(tstats, n - 2) * 2
    pval = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    return cov, cor, slope, intercept, pval, stderr


def date_range(start_date, end_date, increment, period):
    
    """Generate dates seperated by given time increment/period"""
    
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result


# Extract vertex coordinates and heights from geopandas
def contours_to_arrays(gdf, col):

    coords_zvals = []

    for i in range(1, len(gdf)):

        val = gdf.iloc[i][col]

        try:
            coords = np.concatenate([np.vstack(x.coords.xy).T for x in gdf.iloc[i].geometry])

        except:
            coords = np.vstack(gdf.iloc[i].geometry.coords.xy).T

        coords_zvals.append(np.column_stack((coords, np.full(np.shape(coords)[0], fill_value=val))))

    return np.concatenate(coords_zvals)


# Import external dea-notebooks functions using relative link to Scripts directory
sys.path.append('../10_Scripts')
import DEADataHandling, SpatialTools, DEAPlotting, BandIndices

%load_ext autoreload
%autoreload 2

## Set up analysis

In [6]:
# Set coords
# lat, lon, study_area = -14.797361, 128.466634, "cambridge"  
# lat, lon, study_area = -24.75, 152.353044, "burnett" 
# lat, lon, study_area = -15.19733, 141.59417, 'mitchell'
lat, lon, study_area = -15.9448, 124.1996, 'doubtfulbay'
time_range = ('1986-06-01', '2018-12-31')

# Create datacube instance
dc = datacube.Datacube(app='Tidal tagging')

# Set up analysis data query
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - 6000, x + 5000),
         'y': (y - 4000, y + 4500),         
         'crs': 'EPSG:3577',
         'resolution': (-25, 25),
         'output_crs': 'EPSG:3577',
         'time': time_range}

DEAPlotting.display_map(x=query['x'], y=query['y'])

## Load data

In [ ]:
# Load in only clear Landsat observations with < 1% unclear values
data = DEADataHandling.load_clearlandsat(dc=dc, query=query, 
                                       bands_of_interest=['red', 'green', 'blue', 'nir', 'swir1', 'swir2'], 
                                       masked_prop=0.0, mask_pixel_quality=True) 

# Compute water indices
data['NDWI'] = BandIndices.calculate_indices(data, 'NDWI')
# data['MNDWI'] = BandIndices.calculate_indices(data, 'MNDWI')
# data['WI'] = BandIndices.calculate_indices(data, 'WI')
# data['AWEI_noshadow'] = BandIndices.calculate_indices(data, 'AWEI_noshadow')
# data['AWEI_shadow'] = BandIndices.calculate_indices(data, 'AWEI_shadow')

data = data.astype(np.float32)
data.NDWI.isel(time = 5).plot(size=10)
                                                        

## Compute tides

In [ ]:
# Calculate old tide heights
observed_datetimes = data.time.data.astype('M8[s]').astype('O').tolist()
observed_timepoints = [otps.TimePoint(lon, lat, dt) for dt in observed_datetimes]
observed_predictedtides = otps.predict_tide(observed_timepoints)
tideheights_old = [predictedtide.tide_m for predictedtide in observed_predictedtides]
data['tideheights_old'] = xr.DataArray(tideheights_old, [('time', data.time)])


## Rolling NIDEM

In [ ]:
time_period = ('1987-01-01', '2018-01-01')
epoch_years = 10

# Create list of epochs between start and end of time_period in datetime format
start = datetime.strptime(time_period[0], '%Y-%m-%d')
end = datetime.strptime(time_period[1], '%Y-%m-%d')
epochs = date_range(start, end, epoch_years, 'years')

# Print list of epochs
epochs_strings = [epoch.strftime('%Y-%m-%d') for epoch in epochs][:-1]
print('Processing {} epochs: {}'.format(len(epochs_strings), ', '.join(epochs_strings)))


### Interpolation

In [ ]:
# If output data directory doesn't exist, create it
if not os.path.isdir(f'output_data/rolling_outputs/{study_area}/'):
    os.makedirs(f'output_data/rolling_outputs/{study_area}/')
    
# Create output dict
dem_dict = {}
intertidal_mask_dict = {}

for epoch in epochs[:-1]:
    
    # Identify from and to date strings
    from_date = epoch.strftime('%Y-%m-%d')
    to_date = (epoch + relativedelta(years=epoch_years)).strftime('%Y-%m-%d')    
    print(from_date, to_date)   
    

    ###################
    # Convert to tide #
    ###################    

    data_tidal = data.sel(time=slice(from_date, to_date)).swap_dims({'time': 'tideheights_old'}).sortby('tideheights_old')
#     data_tidal = data_tidal[['NDWI', 'MNDWI', 'WI', 'AWEI_noshadow', 'AWEI_shadow', 'red', 'green', 'blue']]
    data_tidal = data_tidal[['NDWI', 'red', 'green', 'blue']]
    data_tidal['tideheights_var'] = data_tidal.tideheights_old

    data_tidal = data_tidal.rolling(tideheights_old = 50, center=True, min_periods=1).median(dim='tideheights_old')
    data_tidal = data_tidal.astype(np.float32)

    # Copy rolling median tide heights back into dimension
    data_tidal['tideheights_old'] = data_tidal['tideheights_var']

    
    ####################
    # Extract contours #
    ####################

    custom_attrs = {'elev_m': data_tidal['tideheights_var'].values.astype(np.float64)}
    custom_attrs_dtypes = {'elev_m': 'float:9.2'}

    nidem_rolling_df = SpatialTools.contour_extract(z_values=0.0,
                                 ds_array=data_tidal.NDWI,
                                 ds_crs=data.geobox.crs,
                                 ds_affine=data.geobox.affine,
                                 output_shp=f'output_data/rolling_outputs/{study_area}/contours_{study_area}_{from_date[0:4]}_{to_date[0:4]}.shp',
                                 dim='tideheights_old',
                                 attribute_data=custom_attrs,
                                 attribute_dtypes=custom_attrs_dtypes,
                                 verbose=False)

    ###############
    # Interpolate #
    ###############

    # Extract x, y and z points for interpolation
    all_contours = contours_to_arrays(gdf=nidem_rolling_df, col='elev_m')
    points_xy = all_contours[:, [1, 0]]
    values_elev = all_contours[:, 2]

    # # Create grid to interpolate into
    x_size, _, upleft_x, _, y_size, upleft_y =  data.geobox.transform[0:6]
    xcols = len(data.x)
    yrows = len(data.y)
    bottomright_x = upleft_x + (x_size * xcols)
    bottomright_y = upleft_y + (y_size * yrows)
    grid_y, grid_x = np.mgrid[upleft_y:bottomright_y:1j * yrows, upleft_x:bottomright_x:1j * xcols]

    # # Interpolate x, y and z values using linear/TIN interpolation
    out = scipy.interpolate.griddata(points_xy, values_elev, (grid_y, grid_x), method='linear')

    # # Apply guassian blur to smooth transitions between z values (optional)
    # from skimage import filters
    # out = filters.gaussian(out, sigma=2)   

    ##########
    # Export #
    ##########

    # Remove water in lowest tide, and land in highest tide 
    # todo: replace with removing any pixel always water, always land
    intertidal_mask = np.where(data_tidal.isel(tideheights_old = 0).NDWI > 0, 1, 
                               np.where(data_tidal.isel(tideheights_old = -1).NDWI < 0, 2, 0))
    out_masked = np.where(intertidal_mask > 0, -9999, out)
    
    # Append to dict
    dem_dict[f'{from_date[0:4]}_{to_date[0:4]}'] = out_masked
    intertidal_mask_dict[f'{from_date[0:4]}_{to_date[0:4]}'] = intertidal_mask

    import rasterio
    kwargs = {'driver': 'GTiff',
             'width': xcols,
             'height': yrows,
             'count': 1,
             'dtype': rasterio.float64,
             'crs': 'EPSG:3577',
             'transform': data.geobox.transform,
             'nodata': -9999}

    with rasterio.open(f'output_data/rolling_outputs/{study_area}/dem_{study_area}_{from_date[0:4]}_{to_date[0:4]}.tif', 'w', **kwargs) as target:
        target.write_band(1, out_masked)
        
        
    #############
    # Geomedian #
    #############
    
    geomedian_layer = data_tidal[['red', 'green', 'blue']].sel(tideheights_old = data_tidal.tideheights_var < 0.0).median(dim='tideheights_old')

    kwargs = {'driver': 'GTiff',
         'width': xcols,
         'height': yrows,
         'count': 3,
         'dtype': rasterio.float32,
         'crs': 'EPSG:3577',
         'transform': data.geobox.transform,
         'nodata': -9999}

    with rasterio.open(f'output_data/rolling_outputs/{study_area}/geomedian_{study_area}_{from_date[0:4]}_{to_date[0:4]}.tif', 'w', **kwargs) as target:
        target.write(geomedian_layer.to_array().values)
        


In [ ]:

import rasterio
kwargs = {'driver': 'GTiff',
         'width': xcols,
         'height': yrows,
         'count': 1,
         'dtype': rasterio.int16,
         'crs': 'EPSG:3577',
         'transform': data.geobox.transform,
         'nodata': -9999}

with rasterio.open(f'output_data/rolling_outputs/{study_area}/intertidal_{study_area}_2007_2017.tif', 'w', **kwargs) as target:
    target.write_band(1, intertidal_mask_dict['2007_2017'].astype(rasterio.int16))
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/intertidal_{study_area}_1997_2007.tif', 'w', **kwargs) as target:
    target.write_band(1, intertidal_mask_dict['1997_2007'].astype(rasterio.int16))
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/intertidal_{study_area}_1987_1997.tif', 'w', **kwargs) as target:
    target.write_band(1, intertidal_mask_dict['1987_1997'].astype(rasterio.int16))  

In [ ]:
min_tide = np.nanmin([np.where(i != -9999, i, np.nan) for i in dem_dict.values()])
max_tide = np.nanmax([np.where(i != -9999, i, np.nan) for i in dem_dict.values()])
tide_fills = [np.array([0, min_tide, max_tide])[i] for i in intertidal_mask_dict.values()]
time1, time2, time3 = [np.where(tide_fills[i] == 0, val, tide_fills[i]) for i, val in enumerate(dem_dict.values())]


import rasterio
kwargs = {'driver': 'GTiff',
         'width': xcols,
         'height': yrows,
         'count': 1,
         'dtype': rasterio.float64,
         'crs': 'EPSG:3577',
         'transform': data.geobox.transform,
         'nodata': -9999}

with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_cm_{study_area}_2minus1.tif', 'w', **kwargs) as target:
    target.write_band(1, time2 - time1)
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_cm_{study_area}_3minus2.tif', 'w', **kwargs) as target:
    target.write_band(1, time3 - time2)
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_cm_{study_area}_3minus1.tif', 'w', **kwargs) as target:
    target.write_band(1, time3 - time1)  
    
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_m3_{study_area}_2minus1.tif', 'w', **kwargs) as target:
    target.write_band(1, (time2 - time1) * (25.0 * 25.0))
    
with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_m3_{study_area}_3minus2.tif', 'w', **kwargs) as target:
    target.write_band(1, (time3 - time2) * (25.0 * 25.0))

with rasterio.open(f'output_data/rolling_outputs/{study_area}/diff_m3_{study_area}_3minus1.tif', 'w', **kwargs) as target:
    target.write_band(1, (time3 - time1) * (25.0 * 25.0))

In [ ]:
xr.DataArray(intertidal_mask_dict['2007_2017'].astype(float)).plot()


In [ ]:
plt.imshow(np.where(data_tidal.isel(tideheights_old = 0).NDWI > 0, 1, 
                    np.where(data_tidal.isel(tideheights_old = -1).NDWI < 0, 2, 0)) )



# | (data_tidal.isel(tideheights_old = -1).NDWI < 0)

In [ ]:
# from datacube.helpers import write_geotiff
# from skimage.filters import threshold_otsu

# test = dict(NDWI=data_tidal.isel(tideheights_old=0)['NDWI'],
#             MNDWI=data_tidal.isel(tideheights_old=0)['MNDWI'],
#             AWEI_shadow=data_tidal.isel(tideheights_old=0)['AWEI_shadow'],
#             AWEI_noshadow=data_tidal.isel(tideheights_old=0)['AWEI_noshadow'],
#             WI=data_tidal.isel(tideheights_old=0)['WI'],
#            )

# # Plot RGB
# rgb = data_tidal.isel(tideheights_old=0)[['red','green', 'blue']]
# rgb.attrs['affine'] = data.affine
# rgb.attrs['crs'] = data.crs
# write_geotiff(f'output_data/indextest_rgb.tif', rgb)

# # Export other indices
# for key, to_write in test.items():
#     otsu = threshold_otsu(to_write.values[np.isfinite((to_write > 0).values)])
#     print(key, otsu)
#     to_write = to_write.to_dataset()
#     to_write.attrs['affine'] = data.affine
#     to_write.attrs['crs'] = data.crs
#     write_geotiff(f'output_data/indextest_{key}.tif', to_write, profile_override={'nodata': -9999})
#     write_geotiff(f'output_data/indextest_{key}_classified.tif', (to_write > 0).astype(np.int32), profile_override={'nodata': -9999})
#     write_geotiff(f'output_data/indextest_{key}_otsu.tif', (to_write > otsu).astype(np.int32), profile_override={'nodata': -9999})
    

In [ ]:
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from matplotlib import cm

# 2-d tests - setup scattered data
x = np.random.rand(100)*4.0-2.0
y = np.random.rand(100)*4.0-2.0
z = x*np.exp(-x**2-y**2)
ti = np.linspace(-2.0, 2.0, 100)
XI, YI = np.meshgrid(ti, ti)

# use RBF
rbf = Rbf(x, y, z, epsilon=2)
ZI = rbf(XI, YI)

# plot the result
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI, cmap=cm.jet)
plt.scatter(x, y, 100, z, cmap=cm.jet)
plt.title('RBF interpolation - multiquadrics')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.colorbar()

In [ ]:
Rbf(y=all_contours[:, 1], x=all_contours[:, 0], z=all_contours[:, 2], epsilon=2)



# points_xy = all_contours[:, [1, 0]]
# values_elev = all_contours[:, 2]

In [ ]:
z

## Testing confidence

In [ ]:
data_tidal.mean(dim='tideheights_old').NDWI.plot(size=10)

In [ ]:
DEAPlotting.rgb(data_tidal.isel(x=slice(300, 500), y=slice(0, 200), tideheights_old=0), aspect=1)

In [ ]:
y, x = -1572144.852608, -378147.238835
data_tidal.sel(y=y, x=x, method='nearest').NDWI.plot()
# .rolling(tideheights_old = 50, center=True, min_periods=1).median(dim='tideheights_old')

In [ ]:
test = data_tidal.sel(y=y, x=x, method='nearest').rolling(tideheights_old = 50, center=True, min_periods=1).median(dim='tideheights_old')
test.sel(tideheights_old = (test.NDWI > -0.4) & (test.NDWI < 0.4))[['NDWI', 'tideheights_var']].to_dataframe().corr(method='spearman').iloc[0, 1]


# cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(x=data_tidal.tideheights_var, 
#                   y=data_tidal.NDWI, lagx=0, lagy=0, first_dim='tideheights_old')

In [ ]:
test.sel(tideheights_old = (test.NDWI > -0.4) & (test.NDWI < 0.4)).tideheights_var

In [ ]:
import scipy

scipy.stats.spearmanr(data_tidal.sel(x=slice(x- 500, x + 500), y=slice(y + 500, y - 500)).NDWI.values, 
                      ((data_tidal.sel(x=slice(x- 500, x + 500), y=slice(y + 500, y - 500)).NDWI.values / data_tidal.sel(x=slice(x- 500, x + 500), y=slice(y + 500, y - 500)).NDWI.values) * data_tidal.tideheights_var.values.reshape((490, 1, 1))), axis=0)

In [ ]:
data_tidal.NDWI.values.shape

In [ ]:
((data_tidal.NDWI.values / data_tidal.NDWI.values) * data_tidal.tideheights_var.values.reshape((490, 1, 1)))

In [ ]:
np.broadcast(data_tidal.NDWI.values, data_tidal.tideheights_var.values.reshape((490, 1, 1)))

In [ ]:
np.expand_dims(data_tidal.tideheights_var.values, 2).shape

In [ ]:
data_tidal.tideheights_var.values.reshape((490, 1, 1)).shape

In [ ]:
DEAPlotting.rgb(data_tidal.sel(x=slice(x- 500, x + 500), y=slice(y + 500, y - 500), tideheights_old=slice(0.6, 1.3)), aspect=1, col='tideheights_old', col_wrap=5)
data_tidal.sel(x=slice(x- 500, x + 500), y=slice(y + 500, y - 500), tideheights_old=slice(0.6, 1.3)).NDWI.plot(col='tideheights_old', col_wrap=8)
# 

In [ ]:
data_tidal.isel(x=slice(300, 500), y=slice(0, 200), tideheights_old=300)

In [ ]:
data_tidal

In [ ]:
data_tidal.isel(x=slice(300, 500), y=slice(0, 200), tideheights_old=slice(305, 307))

In [ ]:
data_tidal['tideheights_old'] = data_tidal['tideheights_var']

## Load WA sea level into xarray format

In [ ]:
# # Open all data
# all_nodes = xr.open_mfdataset(paths=files, concat_dim='file').squeeze('coast_id')

# # Interpolate hourly results to get sea level estimates for exact acquisition time
# all_nodes = all_nodes.interp(time=data.time.data.astype('M8[s]').astype('O').tolist())

# # Select only required timesteps
# all_nodes  = all_nodes[['time', 'id', 'node', 'latitude', 'longitude', 'sealevel']]

# # Re-index to give latitude, longitude and time dimensions
# test = all_nodes.set_index({'file': ['latitude', 'longitude']}).unstack()


In [ ]:
# for i in range(0,len(test.time) - 1):
    
#     # Import as a dataframe with lon, lat, time, sea level columns
#     test_df_subset = test.isel(time=i).sealevel.to_dataframe().dropna(axis=0).reset_index() 

#     from pyproj import Proj, transform
#     xx, yy = transform(Proj(init='epsg:4326'), Proj(init='epsg:3577'), test_df_subset.longitude.values, test_df_subset.latitude.values)
#     test_df_subset['Coordinates'] = list(zip(*transform(Proj(init='epsg:4326'), Proj(init='epsg:3577'), test_df_subset.longitude.values, test_df_subset.latitude.values)))

#     # # Create geopandas dataframe by creating geometry column
#     # test_df_subset['Coordinates'] = list(zip(test_df_subset.longitude, test_df_subset.latitude))
#     test_df_subset['Coordinates'] = test_df_subset['Coordinates'].apply(Point)
#     test_df_subset = gpd.GeoDataFrame(test_df_subset, geometry='Coordinates', crs='EPSG:3577')
#     # # # test_gdf.to_file('test.shp')

#     # Interpolate
#     resolution = 500
#     out = interp_tides(x_coords=test_df_subset['longitude'].values, 
#                        y_coords=test_df_subset['latitude'].values, 
#                        z_coords=test_df_subset[['sealevel']].values, 
#                        sigma=15, resolution=resolution)


#     # Create land sea mask from shapefile
#     test_affine = rasterio.transform.from_bounds(west=xx.min(), 
#                                                  south=yy.min(), 
#                                                  east=xx.max(), 
#                                                  north=yy.max(), width=out.shape[1], height=out.shape[0])

#     coastline_mask = rasterio.features.geometry_mask(coastline_gpd.geometry,
#                                                      out_shape=(resolution, resolution),
#                                                      transform=test_affine,
#                                                      all_touched=False, invert=True)
    
#     print(np.min(out), np.max(out))

#     # Remove land
#     out[coastline_mask] = np.nan
# #     out[out > 2500] = np.nan
# #     out[out < 1500] = np.nan

#     # Plot
#     fig, ax = plt.subplots(1, 2, figsize=(20, 16))
#     test_df_subset.plot(column='sealevel', ax=ax[0])
#     im = ax[1].imshow(out, extent=[upleft_x, bottomright_x, bottomright_y, upleft_y], vmin=-2000, vmax=3000)
#     # fig.colorbar(im)

